In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("seaborn")

# Проверка гипотез

### **Нами были выдвинуты следующие гипотезы:**
1) Квартиры на первом и последнем этажах в среднем дешевле, чем остальные     
2) В среднем в ЮЗАО и СВАО разброс цен на квартиры больше, чем, например, в НАО, СЗАО и ЗАО, так как в первых двух почти     одинакове количество квартир как выше медианного значения, так и ниже, в отличие от последних    
3) Мат.ожидание цены апартов / апартов-студии меньше, чем мат.ожидание цены других типов квартиры    
4) Гипотеза о равенстве матожиданий цен на апартаменты и обычные квартиры    
5) Мат.ожидание цены квартиры с отделкой меньше, чем матожидание цены квартиры без отделки    

## Гипотеза о "скидке за 1-ый/последний этаж"

Для того чтобы проверить гипотезу о том, что $\mu$ цены квартир на 1-ом/последнем этаже меньше, чем $\mu$ цен остальных квартир, воспользуемсй t-статистикой, так как мы не знаем распределения наших данных и дисперсии. 

In [9]:
#открываем файл с данными без пропусков и с добавленной колонкой с ценой за 1 кв.м
data_without_miss = pd.read_csv('data_without_miss.csv') 
data_without_miss

,Округ,Ближайшая станция метро,Время до метро,Метро рядом,Застройщик,ЖК,Класс,Тип квартиры,Этаж,Этажей в доме,Отделка,Количество комнат,Площадь квартиры,Жилая площадь,Площадь кухни,Тип дома,Парковка,Год сдачи,Цена,Цена за 1 кв.м
0,СВАО,Марьина роща,17.0,2,Компания Пионер,«PRIDE (Прайд)»,Премиум,квартира,5.0,22.0,Без отделки,1.0,32.20,20.333629,11.724697,Монолитный,Подземная,2025.0,16106665,500206.987578
1,СЗАО,Трикотажная,4.0,3,СК Резиденс,«ÁLIA (АЛИЯ)»,Бизнес,квартира,2.0,26.0,Неизвестно,1.0,48.06,20.333629,11.724697,Монолитно-кирпичный,"Подземная, гостевая",2023.0,16122800,335472.326259
2,САО,Тимирязевская,3.0,3,MR Group,«SYMPHONY 34 (Симфони 34)»,Премиум,квартира,2.0,36.0,Чистовая,1.0,37.62,12.800000,3.100000,Монолитный,"Подземная, гостевая",2025.0,18472662,491033.014354
3,ЗАО,Фили,5.0,3,Аеон Девелопмент,«River Park Towers Кутузовский (Ривер Парк Тау...,Премиум,квартира,2.0,45.0,Без отделки,1.0,42.80,20.333629,11.724697,Монолитно-кирпичный,Подземная,2026.0,22450632,524547.476636
4,САО,Белорусская,4.0,3,MR Group,«Комплекс апартаментов Slava (Слава)»,Премиум,апартаменты,2.0,16.0,Предчистовая,1.0,47.90,20.333629,11.724697,Монолитно-кирпичный,"Подземная, гостевая",2023.0,25160002,525261.002088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,НАО (Новомосковский),Ольховая,17.0,3,Группа Родина,«Russian Design District (Рашен Дизайн Дистрикт)»,Бизнес,квартира,12.0,13.0,Без отделки,1.0,36.90,12.200000,11.400000,Монолитно-кирпичный,"Подземная, гостевая",2023.0,11011700,298420.054201
1096,НАО (Новомосковский),Рассказовка,2.0,3,Абсолют Недвижимость,«Переделкино Ближнее Город-парк»,Комфорт,квартира,1.0,9.0,Без отделки,2.0,52.40,26.300000,11.724697,"Панельный, монолитный","Отдельная многоуровневая, гостевая",2025.0,11043610,210755.916031
1097,СЗАО,Спартак,5.0,3,ПИК,«Holland park (Холланд парк)»,Комфорт,квартира,2.0,13.0,Без отделки,1.0,34.90,10.600000,16.100000,Монолитный,"Подземная, гостевая",2024.0,11191557,320674.985673
1098,САО,Водники,12.0,3,ДСК-1,«Первый Ленинградский»,Комфорт,квартира,2.0,14.0,Предчистовая,2.0,52.90,25.500000,10.600000,Панельный,"Отдельная многоуровневая, гостевая",2023.0,11148781,210752.003781


В качестве целевой переменной будем использовать цену за 1 кв.м. для того, чтобы исключить влияние таких существенных признаков, как площадь квартиры, жилая площадь и др. Так как брать **среднее по цене** всех квартир вместе было бы некорректно в силу того, что в нашей выборке не так много квартир, расположенных на 1 и/или последнем этажах.

Сформулируем гипотезы:
$$
\begin{aligned}
&H_0: \hspace{2mm} \mu_{1/last} = \mu_{other} \hspace{2mm} \text{(скидки за 1-ый/последний этаж нет)} \\
&H_1: \hspace{2mm} \mu_{1/last} < \mu_{other}  \hspace{2mm} \text{(скидка за 1-ый/последний этаж есть)}
\end{aligned}
$$

                                                    Формула для t-статистики:

$$ t = \frac{\bar{X} - \bar{Y} - \mu_0} {\sqrt{\frac {\hat{\sigma}_x^2}{n}}}$$

$(\frac {n+m}{nm})(\frac {(n-1)\hat{\sigma}_x^2 + (m-1)\hat{\sigma}_y^2}{n+m-2})$

In [ ]:
Найдем 

In [21]:
data_without_miss[(data_without_miss['Этаж'] == 1) | (data_without_miss['Этаж'] == data_without_miss['Этажей в доме'])]['Цена за 1 кв.м']

30      187817.430684
39      239778.335645
74      245018.384354
83      394628.294036
124     662172.340967
            ...      
1003    397000.000000
1042    218361.995565
1045    272902.000000
1083    312500.000000
1096    210755.916031
Name: Цена за 1 кв.м, Length: 75, dtype: float64

In [22]:
data_without_miss[(data_without_miss['Этаж'] != 1) & (data_without_miss['Этаж'] != data_without_miss['Этажей в доме'])]['Цена за 1 кв.м']

0       500206.987578
1       335472.326259
2       491033.014354
3       524547.476636
4       525261.002088
            ...      
1094    303504.986301
1095    298420.054201
1097    320674.985673
1098    210752.003781
1099    219573.996063
Name: Цена за 1 кв.м, Length: 1025, dtype: float64